In [9]:
import pandas as pd
import re
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
comments_df = pd.read_csv('edos_labelled_data.csv') 
le = preprocessing.LabelEncoder()
comments_df["label"] = le.fit_transform(comments_df["label"])
comments_df.head()



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/elliothagyard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,rewire_id,text,label,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",0,train
1,sexism2022_english-16993,"Then, she's a keeper. 😉",0,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,0,train
3,sexism2022_english-13021,woman?,0,train
4,sexism2022_english-966,I bet she wished she had a gun,0,train


In [10]:
comments_train_df = comments_df[comments_df["split"] == "train"]
comments_test_df = comments_df[comments_df["split"] == "test"]
X_train = comments_train_df["text"]
Y_train = comments_train_df["label"]
X_test = comments_test_df["text"]
Y_test = comments_test_df["label"]
X_train.tail()

5271    Supporting toxic men and glorifying toxic male...
5272    Find a girl with common beliefs. I have. They ...
5273    not to mention that she's an outright commie w...
5274    Only if you make it clear you're not looking f...
5278                      Yea. Most trans women hate men.
Name: text, dtype: object

In [11]:
def clean(comments: list[str]) -> list[str]:
    wln = WordNetLemmatizer()
    # remove unicode
    # remove punct
    comments_clean = [comment.encode("ascii", "ignore").decode() for comment in comments]
    comments_clean = list(map(lambda x : x.lower(), comments_clean))
    comments_clean = [re.sub(r'(#\w+|\[user\]|\[url\])', '', comment) for comment in comments_clean]
    translator = str.maketrans('', '', string.punctuation)
    comments_clean = [comment.translate(translator) for comment in comments_clean]
    #comments_clean = [[wln.lemmatize(word.strip()) for word in comment.split()] for comment in comments_clean]

    return comments_clean

In [12]:
def toWordFreqDF(x, y):
    clean_x = clean(x)
    vectorizer = CountVectorizer()
    vec = vectorizer.fit_transform(clean_x)
    frequency_df = pd.DataFrame(vec.toarray(), columns=vectorizer.get_feature_names_out())
    frequency_df['_label'] = y.tolist()
    frequency_df['_label'].tail()
    return frequency_df
train_freq = toWordFreqDF(X_train, Y_train)
test_freq = toWordFreqDF(X_test, Y_test)


In [13]:
sexist_comment_percent = len(Y_train.loc[Y_train== 1]) / len(Y_train) 
common_freq = train_freq.sum().loc[train_freq.sum() >= 5]
sexist = train_freq[train_freq["_label"] == 1].sum().loc[train_freq.sum() >= 5]
ratio  = sexist/common_freq.sort_values()
bad_words = ratio.loc[ratio > 1 - sexist_comment_percent + .05].index

def test_word(sentence : str):
    sexist = 0
    for word in sentence.split():
        if word in bad_words:
            sexist = 1
            
    return sexist

predict = []
print()
for comment in clean(X_test):
    predict.append(test_word(comment))
print(sum(predict))



149


In [14]:
mid_words = ratio.loc[
    (1 - sexist_comment_percent + .05 > ratio) 
    & (ratio > sexist_comment_percent + .1)
].index
print(mid_words)
good_words = ratio.loc[sexist_comment_percent - .05 > ratio].index

def test_word2(sentence : str, badVal, midVal, cutoff):
    sexist = 0
    for word in sentence.split():
        if word in bad_words:
            sexist += 1
        elif word in mid_words:
            sexist += .2
    sexist = 1 if sexist >= 1 else 0
    return sexist

predict2 = []
for comment in clean(X_test):
    predict2.append(test_word2(comment, 1, 0.2, 1))
print(sum(predict2))

Index(['17', '18', '21', '30', '40', '50', '90', 'abortion', 'above',
       'abusive',
       ...
       'women', 'womens', 'work', 'worse', 'worst', 'yelling', 'yet', 'youd',
       'youtube', 'youve'],
      dtype='object', length=446)
239


In [15]:
from sklearn.metrics import classification_report
def eval_predictions(pred):
    print(classification_report(Y_test, pred))

import random
baseline = [0 for i in predict]
baseline2 = [int(random.uniform(0, 1) > .8) for i in predict]

eval_predictions(baseline)
eval_predictions(baseline2)
eval_predictions(predict)
eval_predictions(predict2)

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       789
           1       0.00      0.00      0.00       297

    accuracy                           0.73      1086
   macro avg       0.36      0.50      0.42      1086
weighted avg       0.53      0.73      0.61      1086

              precision    recall  f1-score   support

           0       0.73      0.81      0.77       789
           1       0.29      0.21      0.24       297

    accuracy                           0.64      1086
   macro avg       0.51      0.51      0.50      1086
weighted avg       0.61      0.64      0.62      1086

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       789
           1       0.80      0.40      0.53       297

    accuracy                           0.81      1086
   macro avg       0.80      0.68      0.71      1086
weighted avg       0.81      0.81      0.78      1086

              preci

/Users/elliothagyard/opt/anaconda3/envs/CSC_380/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/elliothagyard/opt/anaconda3/envs/CSC_380/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/elliothagyard/opt/anaconda3/envs/CSC_380/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_